In [23]:
#code by Gao Ziyuan
#CE888 Assignment: Question Answering using Random Forests
#using data in \\mlvm\mlvm\ce888labs\ce888labs\labs\assignment1\*.txt


import numpy as np

word_dict = dict(nil = 0)  # define and initialize the dictionary of words

# function name: ConvertVector
# function: convert the text data to numeric representation
#           store the representation of stories with questions in a matrix
#           store the representation of answers in another matrix
# data_name : list that stores filename of data
# word_dict : dictionary storing words and corresponding numbers
# story_question : two-dimensional matrix (index of story,index of word)
# final_answer : one-dimensional matrix (index of answer)
def ConvertVector(data_name,word_dict):
    len_words =0               # define maximum length of words
    len_sentences =0           # define the maximum length of sentences
    q_signal= False            # define the signal of whether there is a question in this line
    
    # initialize the two-dimensional matrix including stories and questions
    story_question = np.zeros((5000,1000*len(data_name)),np.int32)
    # initialize the one-dimensional matrix including answers
    final_answer = np.zeros(1000*len(data_name),np.int32)

    story_num = -1             # define index of story in the data
    sentence_num = -1          # define index of sentence in a story
    question_num = -1          # define index of question in the data
    line_num = -1              # define a signal to check whether next line is the start of a new story

    for i in range(len(data_name)):
        with open (data_name[i]) as f:   # open the data file in order
            for line in f:               # load every line
                line_num += 1
                datalist = line.split()  # split the whole line into many strings
                # When the line number is 1 or the last line includes questions,
                # The index of story increments by 1 and the index of sentence initializes.
                if (datalist[0] == '1') or (line_num == -1):
                    story_num += 1
                    sentence_num = -1
                sentence_num += 1 # index of sentence increments all the time
                # check whether it is the quesion line
                if line.find('?') == -1: 
                    q_signal = False
                else:
                    q_signal = True
                    question_num += 1 # increment of index of question 
                    line_num = -2 # initialization of the signal
            
                for j in range(1,len(datalist)): # Load every word in order in a line
                    word = datalist[j].lower() # Lower every word
                    # Remove the punctuation of the word 
                    if word[-1] == '.' or word[-1] == '?':
                        word = word[0:-1]
                    len_words = max(len_words,j)
                    # Save the words in the data into the dictionary
                    if not(word_dict.has_key(word)):
                        word_dict[word] = len(word_dict)
                    # Give story_question matrix the integer value in the dictionary corresponding to the word
                    story_question[(j-1)+20*sentence_num,story_num] = word_dict[word]
                    if (q_signal):
                        if datalist[j][-1] =='?':
                            ans =datalist[j+1].lower() # Lower the answers
                            # Save the words in the data into the dictionary
                            if not(word_dict.has_key(ans)):
                                word_dict[ans] = len(word_dict)
                            # Put the numeric representation of answers into final_answer matrix
                            final_answer[question_num] = word_dict[ans]
                            break # Skip the loop and remove parameter 'idx'  
                len_sentences = max(len_sentences,sentence_num+1)

    # Transposition  of  story_question matrix            
    story_question = np.transpose(story_question[0:len_words+20*len_sentences, 0:story_num+1])
    return (story_question,final_answer)
                                              

In [24]:
from glob import glob

# Load training datasets
train_data_name = [glob('qa' + str(t) + '_*_train.txt')[0] for t in xrange(1, 21) if t!=2 and t!=3 and t!=5 and t!=8]
train_story_question, train_final_answer = ConvertVector(train_data_name,word_dict)

print (train_story_question)
print (train_final_answer)

# Load testing datasets
test_data_name = [glob('qa' + str(t) + '_*_test.txt')[0] for t in xrange(1, 21) if t!=2 and t!=3 and t!=5 and t!=8 ]
test_story_question, test_final_answer = ConvertVector(test_data_name,word_dict)

print (test_story_question)
print (test_final_answer) 

[[  1   2   3 ...,   0   0   0]
 [ 11   7  12 ...,   0   0   0]
 [  6   2   3 ...,   0   0   0]
 ..., 
 [147  33   4 ...,   0   0   0]
 [154  10 159 ...,   0   0   0]
 [158  37   4 ...,   0   0   0]]
[  5   8   8 ..., 148  19 155]
[[  6  17   3 ...,   0   0   0]
 [ 11   7  12 ...,   0   0   0]
 [  6   7   3 ...,   0   0   0]
 ..., 
 [147  10 155 ...,   0   0   0]
 [154  40   4 ...,   0   0   0]
 [147  17   3 ...,   0   0   0]]
[  8   5  19 ...,  14 159 155]


In [25]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier



X_train = train_story_question
Y_train = train_final_answer
# Tuple of array dimensions
print X_train.shape
print Y_train.shape


X_test = test_story_question
Y_test = test_final_answer
# Tuple of array dimensions
print X_test.shape
print Y_test.shape

# Build the model of RandomForest (set number of trees as 100)
clf = RandomForestClassifier(n_estimators=100)
# Train the model by using training datasets
clf = clf.fit(X_train,Y_train)
# Test the model by using testing datasets
score = clf.score(X_test,Y_test)

(16000, 551)
(16000,)
(16000, 551)
(16000,)
0.5391875
